In [4]:
import os, shutil, random
from pathlib import Path

BASE = Path(r"D:\psm2\BodyPartRaw")       # your raw images
OUT  = Path(r"D:\psm2\BodyPartDataset")   # output with train/val/test

CLASSES = ["face", "hand", "leg"]
SPLITS = {"train":0.7, "val":0.2, "test":0.1}

for cls in CLASSES:
    files = [f for f in (BASE/cls).iterdir() if f.is_file()]
    random.shuffle(files)
    n = len(files)
    n_train = int(SPLITS["train"]*n)
    n_val   = int(SPLITS["val"]*n)
    
    split_map = {
        "train": files[:n_train],
        "val":   files[n_train:n_train+n_val],
        "test":  files[n_train+n_val:]
    }

    for split, flist in split_map.items():
        outdir = OUT/split/cls
        outdir.mkdir(parents=True, exist_ok=True)
        for f in flist:
            shutil.copy(f, outdir/f.name)

print("✅ Done splitting BodyPartDataset")


✅ Done splitting BodyPartDataset


In [5]:
# train_bodypart_vgg16.py
import os, json, collections, tensorflow as tf
from tensorflow.keras.applications import vgg16
from tensorflow.keras import layers, models, callbacks, optimizers

DATA = r"D:\psm2\BodyPartDataset"
OUT  = r"D:\psm2\models"; os.makedirs(OUT, exist_ok=True)
IMG = (224,224); BATCH = 32

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, rotation_range=15, width_shift_range=0.08, height_shift_range=0.08,
    zoom_range=0.15, shear_range=0.08, horizontal_flip=True)
val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(f"{DATA}\\train", target_size=IMG, batch_size=BATCH, class_mode="categorical")
val_ds   = val_gen.flow_from_directory(f"{DATA}\\val",   target_size=IMG, batch_size=BATCH, class_mode="categorical")

with open(os.path.join(OUT,"bodypart_class_indices.json"),"w") as f:
    json.dump(train_ds.class_indices, f, indent=2)

# class weights (helps if 'leg' is small)
counts = collections.Counter(train_ds.classes.tolist())
mx = max(counts.values()); class_weight = {i: mx / counts.get(i,1) for i in range(train_ds.num_classes)}

base = vgg16.VGG16(weights="imagenet", include_top=False, input_shape=IMG+(3,))
base.trainable = False
model = models.Sequential([ base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.35),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.35),
    layers.Dense(train_ds.num_classes, activation="softmax") ])

model.compile(optimizer=optimizers.Adam(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
ckp = callbacks.ModelCheckpoint(r"D:\psm2\models\bodypart_vgg16.h5", save_best_only=True, monitor="val_accuracy")
es  = callbacks.EarlyStopping(patience=6, restore_best_weights=True)
rlr = callbacks.ReduceLROnPlateau(patience=3, factor=0.3)

model.fit(train_ds, epochs=20, validation_data=val_ds, callbacks=[ckp, es, rlr], class_weight=class_weight)

# fine-tune last block
base.trainable = True
for layer in base.layers[:-4]: layer.trainable = False
model.compile(optimizer=optimizers.Adam(1e-4), loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[ckp, es, rlr], class_weight=class_weight)
print("saved model to D:\\psm2\\models\\bodypart_vgg16.h5")


Found 518 images belonging to 3 classes.
Found 148 images belonging to 3 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


C:\Users\Fazreen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


C:\Users\Fazreen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5700 - loss: 2.8011

...

17/17 ━━━━━━━━━━━━━━━━━━━━ 84s 5s/step - accuracy: 0.8181 - loss: 1.2313 - val_accuracy: 0.9527 - val_loss: 0.2390 - learning_rate: 0.0010
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 83s 5s/step - accuracy: 0.8957 - loss: 1.0721 - val_accuracy: 0.9257 - val_loss: 0.2596 - learning_rate: 0.0010
Epoch 6/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8934 - loss: 0.8943

17/17 ━━━━━━━━━━━━━━━━━━━━ 84s 5s/step - accuracy: 0.8939 - loss: 0.8903 - val_accuracy: 0.9595 - val_loss: 0.1406 - learning_rate: 0.0010
Epoch 7/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 85s 5s/step - accuracy: 0.9328 - loss: 0.7053 - val_accuracy: 0.9257 - val_loss: 0.1983 - learning_rate: 0.0010
Epoch 8/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 86s 5s/step - accuracy: 0.8875 - loss: 0.6418 - val_accuracy: 0.9392 - val_loss: 0.1489 - learning_rate: 0.0010
Epoch 9/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 111s 7s/step - accuracy: 0.9248 - loss: 0.5136 - val_accuracy: 0.9257 - val_loss: 0.1781 - learning_rate: 0.0010
Epoch 10/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 114s 7s/step - accuracy: 0.8866 - loss: 0.5973 - val_accuracy: 0.9459 - val_loss: 0.1297 - learning_rate: 3.0000e-04
Epoch 11/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 121s 7s/step - accuracy: 0.9076 - loss: 0.6265 - val_accuracy: 0.9459 - val_loss: 0.1207 - learning_rate: 3.0000e-04
Epoch 12/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 147s 9s/step - accuracy: 0.9475 - loss: 0.3995 - val_accura

17/17 ━━━━━━━━━━━━━━━━━━━━ 126s 7s/step - accuracy: 0.9312 - loss: 0.3860 - val_accuracy: 0.9662 - val_loss: 0.0966 - learning_rate: 3.0000e-04
Epoch 16/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 132s 8s/step - accuracy: 0.9418 - loss: 0.4015 - val_accuracy: 0.9662 - val_loss: 0.0932 - learning_rate: 3.0000e-04
Epoch 17/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 137s 8s/step - accuracy: 0.9351 - loss: 0.3907 - val_accuracy: 0.9595 - val_loss: 0.0992 - learning_rate: 3.0000e-04
Epoch 18/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 134s 8s/step - accuracy: 0.9152 - loss: 0.4928 - val_accuracy: 0.9662 - val_loss: 0.1010 - learning_rate: 3.0000e-04
Epoch 19/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 122s 7s/step - accuracy: 0.9273 - loss: 0.4973 - val_accuracy: 0.9595 - val_loss: 0.1045 - learning_rate: 3.0000e-04
Epoch 20/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 129s 8s/step - accuracy: 0.9441 - loss: 0.3638 - val_accuracy: 0.9595 - val_loss: 0.1038 - learning_rate: 9.0000e-05
Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8520 - loss: 

17/17 ━━━━━━━━━━━━━━━━━━━━ 151s 8s/step - accuracy: 0.8536 - loss: 1.0386 - val_accuracy: 0.9865 - val_loss: 0.0680 - learning_rate: 1.0000e-04
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 141s 8s/step - accuracy: 0.9876 - loss: 0.1550 - val_accuracy: 0.9865 - val_loss: 0.1073 - learning_rate: 1.0000e-04
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 150s 9s/step - accuracy: 0.9797 - loss: 0.0847 - val_accuracy: 0.9865 - val_loss: 0.1196 - learning_rate: 1.0000e-04
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 157s 9s/step - accuracy: 0.9819 - loss: 0.0991 - val_accuracy: 0.9662 - val_loss: 0.1141 - learning_rate: 1.0000e-04
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9869 - loss: 0.0916

17/17 ━━━━━━━━━━━━━━━━━━━━ 149s 9s/step - accuracy: 0.9871 - loss: 0.0901 - val_accuracy: 0.9932 - val_loss: 0.1442 - learning_rate: 3.0000e-05
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 142s 8s/step - accuracy: 0.9917 - loss: 0.0311 - val_accuracy: 0.9932 - val_loss: 0.1463 - learning_rate: 3.0000e-05
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 145s 9s/step - accuracy: 0.9998 - loss: 0.0071 - val_accuracy: 0.9865 - val_loss: 0.1167 - learning_rate: 3.0000e-05
saved model to D:\psm2\models\bodypart_vgg16.h5
